In [ ]:
using Parameters

struct ProdBPProblem
    bpproblem::BPProblem
    Imax::Int
    alpha :: Float64
    Le :: Matrix{Float64}
    La :: Matrix{Float64}
    Lapp :: Matrix{Float64}
    Lch :: Matrix{Float64}
end

ProdBPProblem(N,K,mi,Imax,alpha) = begin
    t = BPProblem(N,K,mi)
    n = Int(log2(N))
    Le = zeros(N,N)
    La = zeros(N,N)
    Lapp = zeros(N,N)
    Lch = zeros(N,N)
    ProdBPProblem(t,Imax,alpha,Le,La,Lapp,Lch)
end

abstract type DIRECTION end
struct ROW <: DIRECTION
    i::Int
end
struct COLUMN <: DIRECTION
    i::Int
end

function initialize!(t::ProdBPProblem,ch::AWGN)
    updatellr!(ch)
    t.La .= zero(eltype(t.La))
    t.Le .= zero(eltype(t.Le))
    t.Lapp .= zero(eltype(t.Lapp))
    #@views t.Lch[:] .= ch.llr
    for i=1:ch.N
       t.Lch[i] = ch.llr[i]
    end
    nothing
end

function initialize!(i,::Type{ROW},t::ProdBPProblem)
    x = t.bpproblem; F = x.polar.F
    @views initializeLandR!(x.L,x.R,F,t.Lch[i,:])
    @views x.L[:,end] .=  x.L[:,end] .+ t.La[i,:]
    nothing
end

function initialize!(i,::Type{COLUMN},t::ProdBPProblem)
    x = t.bpproblem; F = x.polar.F
    @views initializeLandR!(x.L,x.R,F,t.Lch[:,i])
    @views x.L[:,end] .=  x.L[:,end] .+ t.La[:,i]
    nothing
end

function update!(i,::Type{ROW},t::ProdBPProblem)
    x = t.bpproblem
    for k=1:x.N
        t.Lapp[i,k] = x.L[k,end] + x.R[k,end]
    end
    nothing
end

function update!(i,::Type{COLUMN},t::ProdBPProblem)
    x = t.bpproblem
    for k=1:x.N
        t.Lapp[k,i] = x.L[k,end] + x.R[k,end]
    end
    nothing
end

function update!(t::ProdBPProblem)
    t.Le .= t.Lapp .- t.Lch .- t.La
    t.La .= t.alpha .*t.Le
    nothing
end


function errors(t::ProdBPProblem)
    A = t.s.polar.A
    sum(ifelse(t.Lapp[i,j] < 0.0,1,0) for i in A, j in A) 
end

function solver(sim,w::ProdBPProblem, ch::AWGN)
    @unpack bpproblem,Imax,alpha,Le,La,Lapp,Lch = w
    t = bpproblem; F = t.polar.F
    n = Int(log2(t.N))
    v = zeros(Int,t.N); u = zeros(Int,t.N)
    function checkcond()
        for i=1:t.N
            v[i] = harddecision(Int,t.L[i,end],t.R[i,end])
            u[i] = harddecision(Int,t.L[i,1],t.R[i,1])
        end
        Gcheck(u,v)
    end

    bler = 0; ber = 0; sucnt = 0; error = Int[]
    for s = 1:sim
        # LLRの更新 Lapp,La,Le,Lch等の行列の初期化
        initialize!(w,ch)
        
        #
        for _=1:Imax
            count = 0
            for direction in (ROW,COLUMN)
                count = 0
                for i=1:t.N
                    initialize!(i,direction,w)
                    success = BP(t.N,t.mi,t.L,t.R,t.M,checkcond)
                    count += ifelse(success,0,1)
                    update!(i,direction,w)
                end
                update!(w)
                if iszero(count) ;sucnt += 1; break; end
            end
            if iszero(count) ; break; end
        end
        tmp = errors(t)
        push!(error,tmp)
        ber += tmp
        @views if !isnothing(findfirst(<(0.0),w.Lapp[t.polar.A,t.polar.A]))
            bler += 1
        end
    end
    return bler/sim, ber/sim/ch.K, sucnt, error
end

In [ ]:
function initialize!()
    
end